In [9]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Corpus creation 

-Requirements
    oTokenization 
        Translate sentences into meaningful blocks
    oBuild Bag-of-words
        Vectorized dictionary
-Algorithms to explore
    oSkip-gram
    oWord-2-vec
    oTf-idf
    oHierachical Softmax
    oNegative Sampling
    oNaïve Softmax
    oMorphological segmentation
    oPart-of-speech tagging
    oSentence breaking
        -Topic detection 
    oRequirements
        Stop Words
        Regular expression tokenizer
        Snowball stemmer
    oAlgorithms to consider
        Latent Dirichlet Allocation (LDA) topic
        Skip Gram (context prediction) 
        Multinominal Naïve Bayes


In [3]:
import pandas as pd
from nltk.tokenize import sent_tokenize

In [4]:
# df = pd.read_excel("HANSARDfullDataset.xlsx")
df = pd.read_excel("HansardFlatDatasetUpdated.xlsx")

In [5]:
df.head()

,qTextID,qHeading,talkerID,talkKind,qDate,qNum,houseName,date,sessionName,parliamentNum,...,discussionType,hqTextID,tqTextID,qtalkerID,qtalkKind,textID,talkerRole,talkerName,talkerTranscript,fileName
0,201904304784844826e74509a0000001,Dog Fence,1820,answer,2019-04-30,NaN,Legislative Council,2019-04-30,"Fifty-Fourth Parliament, First Session (54-1)",54,...,Answers to Questions,201904304784844826e74509a0000001,201904304784844826e74509a0000001,3404,question,201904304784844826e74509a0000004,member,The Hon. D.W. RIDGWAY,1.&#x9;I refer the honourable member to the me...,HANSARD-10-26496.xml
1,201904304784844826e74509a0000001,Dog Fence,1820,answer,2019-04-30,NaN,Legislative Council,2019-04-30,"Fifty-Fourth Parliament, First Session (54-1)",54,...,Answers to Questions,201904304784844826e74509a0000001,201904304784844826e74509a0000001,3404,question,201904304784844826e74509a0000005,member,The Hon. D.W. RIDGWAY,"2.&#x9;In total, 115 kilometres of the 2,150 k...",HANSARD-10-26496.xml
2,201904304784844826e74509a0000001,Dog Fence,1820,answer,2019-04-30,NaN,Legislative Council,2019-04-30,"Fifty-Fourth Parliament, First Session (54-1)",54,...,Answers to Questions,201904304784844826e74509a0000001,201904304784844826e74509a0000001,3404,question,201904304784844826e74509a0000006,member,The Hon. D.W. RIDGWAY,3.&#x9;Based on the advice received about the ...,HANSARD-10-26496.xml
3,2019043047e58af38d714803b0000002,Adelaide Metro,3404,question,2019-04-30,NaN,Legislative Council,2019-04-30,"Fifty-Fourth Parliament, First Session (54-1)",54,...,Answers to Questions,2019043047e58af38d714803b0000002,2019043047e58af38d714803b0000002,3404,question,2019043047e58af38d714803b0000003,member,The Hon. J.A. DARLEY,In reply to the Hon. J.A. DARLEY (5 December 2...,HANSARD-10-26492.xml
4,2019043047e58af38d714803b0000002,Adelaide Metro,1820,answer,2019-04-30,NaN,Legislative Council,2019-04-30,"Fifty-Fourth Parliament, First Session (54-1)",54,...,Answers to Questions,2019043047e58af38d714803b0000002,2019043047e58af38d714803b0000002,3404,question,2019043047e58af38d714803b0000004,member,The Hon. D.W. RIDGWAY,"The Minister for Transport, Infrastructure a...",HANSARD-10-26492.xml


In [6]:
dfTemp = df[['qTextID', 'talkerTranscript']]
dfTemp = dfTemp.dropna()

In [22]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xcelsior\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Sentence Tokenize

In [26]:
dfTemp = df[['qtalkKind',
       'textID', 'talkerRole', 'talkerName', 'talkerTranscript']]

sentTokens = []

for i in dfTemp['talkerTranscript']:
    try:
        a = sent_tokenize(i)
        sentTokens.append(a)
        print(a)
    except:
        print("error")
        sentTokens.append("blank")


["1.&#x9;I refer the honourable member to the media release issued on 7 March 2019 titled '$25 Million fix required for Ageing Dog Fence.'"]
['2.&#x9;In total, 115 kilometres of the 2,150 kilometre South Australian Dog Fence was replaced over 16\xa0years under the former Labor government.', "Of this, 68.5 kilometres is electric fence, which now needs to be replaced because it is too low and because pastoralists don't believe it is effective."]
['3.&#x9;Based on the advice received about the failure of the electric fencing installed under the former Labor government I would not envisage replacing sections of the Dog Fence with electric fencing.']
['In reply to the Hon.', 'J.A.', 'DARLEY (5 December 2018).']
['  The Minister for Transport, Infrastructure and Local Government has provided the following advice:The Hon.', 'D.W. RIDGWAY (Minister for Trade, Tourism and Investment):  The Minister for Transport, Infrastructure and Local Government has provided the following advice:']
['Upkeep 

error
error
['Human Services']
error
error
['Innovation and Skills']
error
error
['Police, Emergency Services and Correctional Services']
error
error
['Primary Industries and Regional Development']
error
error
['Trade, Tourism and Investment']
error
error
['Transport, Infrastructure and Local Government']
error
error
['Treasurer']
error
error
['Parliament']
error
error
['Premier/Education/Innovation and Skills/Trade, Tourism and Investment']
error
error
['South Australian Government Capital Program as at 2018-19 Budget ($000s)']
['Premier']
error
error
['Attorney-General']
error
error
['Child Protection']
error
error
['Education']
error
error
['Energy and Mining']
error
error
['Environment and Water']
error
error
['Health and Wellbeing']
error
error
['Human Services']
error
error
['Innovation and Skills']
error
error
['Police, Emergency Services and Correctional Services']
error
error
['Primary Industries and Regional Development']
error
error
['Trade, Tourism and Investment']
error
er

['I refer the member to my answer to question with notice number 939.']
['926 Ms STINSON (Badcoe) (2 July 2019).', 'Can the Minister for Child Protection advise:']
['(a)&#x9;What consultancies have been engaged by the Department for Child Protection since 18\xa0March\xa02018?']
['(b)&#x9;What is the purpose of each consultancy engaged by the Department for Child Protection since 18\xa0March 2018?']
['(c)&#x9;What is the cost of each consultancy engaged by the Department for Child Protection since 18\xa0March 2018?']
['  I have been advised:The Hon.', 'R. SANDERSON (Adelaide—Minister for Child Protection):  I have been advised:']
['The Department for Child Protection has not yet finalised its financial reporting for 2018-19, therefore this data is not currently available.', "Reporting on consultancies is made publicly available each year as part of the department's Annual Report."]
['In 2017-18, for consultancies above $10,000 , information can be found here: https://data.sa.gov.au/data

['122 Dr CLOSE (Port Adelaide—Deputy Leader of the Opposition) (24 July 2018).', "Can the minister guarantee that Elizabeth Vale Primary School will receive $7.5 million under the Building Better Schools Infrastructure Program and be able to progress with plans to build new flexible learning spaces and special education spaces, and redevelop the administration area and school entrance, as outlined on the Department for Education and Child Development's website?"]
['  I have been advised of the following:The Hon.', 'J.A.W.', 'GARDNER (Morialta—Minister for Education):  I have been advised of the following:']
["The government proposes to deliver this school's Building Better Schools project in line with the announced dollar value as described."]
["The illustrations and lists of desired improvements prepared for all schools in the program clearly stated 'These initial plans are subject to further planning and may change'.", 'This statement was placed under the illustration when the former

['  I have been advised:The Hon.', 'C.L.', 'WINGARD (Gibson—Minister for Police, Emergency Services and Correctional Services, Minister for Recreation, Sport and Racing):  I have been advised:']
['The Department for Correctional Services (DCS) is focused on increasing the number of female staff in correctional officer positions including leadership positions.']
['The DCS Strategic Workforce Plan 2017 – 2020 contains a strategy of Implementing initiatives for recruitment of female Trainee Correctional Officers (TCOs).']
['The target for the strategy is 50 per cent and actions include:']
['Working with government and non-government organisations in the promotion of the role.']
['Focussing on attracting female recruits through employment expos and information sessions.']
['Exploring an increase in the uptake of flexible working arrangements.']
['One aspect of the Better Prisons Program is the recruitment of part time and casual correctional officers and DCS consider that these options wil

['The first variation is outlined in Budget Paper 4, Volume 2, page 54.']
["The increase in 'Other non-current liabilities' relates to the lease incentive associated with the new lease for head office accommodation for the Electoral Commission."]
['100 The Hon.', 'K.J.', 'MAHER (Leader of the Opposition) (25 October 2018).', 'Can the Attorney-General advise:']
['What sections are the additional 8 FTEs for SACAT going into?', 'What level will they be, what role or function will they fulfil, how many FTEs are currently engaged and, does the Attorney-General expect to fill all FTE positions?']
['  The Attorney-General has provided the following advice:The Hon.', 'R.I. LUCAS (Treasurer):  The Attorney-General has provided the following advice:']
['SACAT has been provided further resources for additional jurisdictions that are expected to transition into SACAT, together with resources for accommodation changes.', 'There has also been an increase in indirect costs nominally allocated to this

['As at 18 February 2019, $57.9 million in guarantee fees have been paid by SA Water to SAFA for the 2018-19\xa0financial year.', 'The guarantee fee rate for the 2018-19 financial year in percentage terms is 1.36 per cent per annum.']
['660 The Hon.', 'S.C. MULLIGHAN (Lee) (27 February 2019).', 'How much of the $150 million TVSP\xa0scheme announced in the 2018-19 budget been spent in total?']
['1.&#x9;How many FTEs have accepted in total?']
['2.&#x9;How many FTEs accepted per department?']
['3.&#x9;What was the classification level of FTE recipients, by department?']
['4.&#x9;What is the average cost per FTE accepted?']
['  I have been advised:The Hon.', 'S.S. MARSHALL (Dunstan—Premier):  I have been advised:']
['1.', '& 2.&#x9;To the end of January 2019, agencies had sought reimbursement from the Department of Treasury and Finance for costs of $18 million (169.9 FTE separations) against the central TVSP provision.', 'Reductions in FTE numbers in the public sector are not solely relian

In [10]:
df.columns

Index(['qTextID', 'qHeading', 'talkerID', 'talkKind', 'qDate', 'qNum',
       'houseName', 'date', 'sessionName', 'parliamentNum', 'sessionNum',
       'parliamentName', 'reviewStage', 'dateModified', 'proceeding',
       'discussionType', 'hqTextID', 'tqTextID', 'qtalkerID', 'qtalkKind',
       'textID', 'talkerRole', 'talkerName', 'talkerTranscript', 'fileName'],
      dtype='object')

## Word Tokenize - Treebank tokenization

In [37]:
from nltk.tokenize import word_tokenize

wordTokens = []
for i in sentTokens:
    a = word_tokenize(i[0])
    print(a)
    wordTokens.append(a)

['1.', '&', '#', 'x9', ';', 'I', 'refer', 'the', 'honourable', 'member', 'to', 'the', 'media', 'release', 'issued', 'on', '7', 'March', '2019', 'titled', "'", '$', '25', 'Million', 'fix', 'required', 'for', 'Ageing', 'Dog', 'Fence', '.', "'"]
['2.', '&', '#', 'x9', ';', 'In', 'total', ',', '115', 'kilometres', 'of', 'the', '2,150', 'kilometre', 'South', 'Australian', 'Dog', 'Fence', 'was', 'replaced', 'over', '16', 'years', 'under', 'the', 'former', 'Labor', 'government', '.']
['3.', '&', '#', 'x9', ';', 'Based', 'on', 'the', 'advice', 'received', 'about', 'the', 'failure', 'of', 'the', 'electric', 'fencing', 'installed', 'under', 'the', 'former', 'Labor', 'government', 'I', 'would', 'not', 'envisage', 'replacing', 'sections', 'of', 'the', 'Dog', 'Fence', 'with', 'electric', 'fencing', '.']
['In', 'reply', 'to', 'the', 'Hon', '.']
['The', 'Minister', 'for', 'Transport', ',', 'Infrastructure', 'and', 'Local', 'Government', 'has', 'provided', 'the', 'following', 'advice', ':', 'The', 'Ho

['802', 'The', 'Hon', '.']
['1.', '&', '#', 'x9', ';', '$', '0', 'to', '$', '200,000']
['2.', '&', '#', 'x9', ';', '$', '200,001', 'to', '$', '300,000', ';']
['3.', '&', '#', 'x9', ';', '$', '300,001', 'to', '$', '400,000', ';']
['4.', '&', '#', 'x9', ';', '$', '400,001', 'to', '$', '500,000', ';']
['5.', '&', '#', 'x9', ';', '$', '500,000', 'to', '$', '600,000', ';']
['6.', '&', '#', 'x9', ';', '$', '600,001', 'to', '$', '700,000', ';']
['7.', '&', '#', 'x9', ';', '$', '700,001', 'to', '$', '800,000', ';']
['8.', '&', '#', 'x9', ';', '$', '800,001', 'to', '$', '900,000', ';']
['9.', '&', '#', 'x9', ';', '$', '900,001to', '$', '1,000,000', ';']
['10.', '&', '#', 'x9', ';', '$', '1,000,001', 'to', '$', '1,100,000', ';']
['11.', '&', '#', 'x9', ';', '$', '1,100,001', 'to', '$', '1,200,000', ';']
['12.', '&', '#', 'x9', ';', '$', '1,200,001', 'to', '$', '1,300,000', ';']
['13.', '&', '#', 'x9', ';', '$', '1,300,001', 'to', '$', '1,400,000', ';']
['14.', '&', '#', 'x9', ';', '$', '1,400,00

['840', 'Mr', 'PICTON', '(', 'Kaurna', ')', '(', '4', 'June', '2019', ')', '.']
['(', 'a', ')', '&', '#', 'x9', ';', 'Chief', 'Executive', ',', 'Department', 'of', 'the', 'Premier', 'and', 'Cabinet', '?']
['(', 'b', ')', '&', '#', 'x9', ';', 'Chief', 'Executive', ',', 'Department', 'of', 'Treasury', 'and', 'Finance', '?']
['(', 'c', ')', '&', '#', 'x9', ';', 'Chief', 'Executive', ',', 'Department', 'of', 'Human', 'Services', '?']
['(', 'd', ')', '&', '#', 'x9', ';', 'Chief', 'Executive', ',', 'Department', 'for', 'Education', '?']
['(', 'e', ')', '&', '#', 'x9', ';', 'Chief', 'Executive', ',', 'Department', 'for', 'Health', 'and', 'Wellbeing', '?']
['(', 'f', ')', '&', '#', 'x9', ';', 'Chief', 'Executive', ',', 'Attorney-General', "'s", 'Department', '?']
['(', 'g', ')', '&', '#', 'x9', ';', 'Chief', 'Executive', ',', 'Department', 'for', 'Innovation', 'and', 'Skills', '?']
['(', 'h', ')', '&', '#', 'x9', ';', 'Chief', 'Executive', ',', 'Department', 'of', 'Planning', ',', 'Transport',

['I', 'have', 'been', 'advised', ':', 'The', 'Hon', '.']
['Although', 'the', 'funding', 'procedure', 'was', 'under', 'review', ',', 'applications', 'were', 'still', 'considered', 'on', 'a', 'case-by-case', 'basis', '.']
['969', 'Ms', 'STINSON', '(', 'Badcoe', ')', '(', '19', 'June', '2019', ')', '.']
['I', 'have', 'been', 'advised', ':', 'The', 'Hon', '.']
['The', 'Department', 'for', 'Child', 'Protection', 'has', 'not', 'yet', 'finalised', 'its', 'financial', 'reporting', 'for', '2018-19', ',', 'therefore', 'this', 'data', 'is', 'not', 'currently', 'available', '.']
['975', 'Ms', 'STINSON', '(', 'Badcoe', ')', '(', '19', 'June', '2019', ')', '.']
['I', 'have', 'been', 'advised', ':', 'The', 'Hon', '.']
['The', 'Department', 'for', 'Child', 'Protection', 'has', 'not', 'yet', 'finalised', 'its', 'financial', 'reporting', 'for', '2018-19', ',', 'therefore', 'this', 'data', 'is', 'not', 'currently', 'available', '.']
['895', 'Ms', 'STINSON', '(', 'Badcoe', ')', '(', '18', 'June', '2019', 

['154', 'Dr', 'CLOSE', '(', 'Port', 'Adelaide—Deputy', 'Leader', 'of', 'the', 'Opposition', ')', '(', '24', 'July', '2018', ')', '.']
['I', 'have', 'been', 'advised', 'of', 'the', 'following', ':', 'The', 'Hon', '.']
['The', 'government', 'proposes', 'to', 'deliver', 'this', 'school', "'s", 'Building', 'Better', 'Schools', 'project', 'in', 'line', 'with', 'the', 'announced', 'dollar', 'value', 'as', 'described', '.']
['The', 'illustrations', 'and', 'lists', 'of', 'desired', 'improvements', 'prepared', 'for', 'all', 'schools', 'in', 'the', 'program', 'clearly', 'stated', "'These", 'initial', 'plans', 'are', 'subject', 'to', 'further', 'planning', 'and', 'may', 'change', "'", '.']
['The', 'work', 'currently', 'being', 'undertaken', 'by', 'the', 'Department', 'for', 'Education', 'in', 'collaboration', 'with', 'this', 'school', 'will', 'determine', 'the', 'detail', 'of', 'the', 'final', 'project', '(', 'or', 'projects', ')', 'that', 'will', 'go', 'ahead', '.']
['the', 'removal', 'of', 'old

...

['Total', 'number', 'of', 'clients', 'housed', 'through', 'Housing', 'SA', 'at', '31', 'March', '2018', 'on', ':']
['b']
['b']
['*Data', 'is', 'recorded', 'as', 'at', 'the', 'end', 'of', 'each', 'month', '.']
['The', 'remainder', 'of', 'tenants', 'were', 'either', 'housed', 'in', 'low', 'demand', 'areas', 'or', 'through', 'the', 'Specialised', 'Housing', 'Program', ',', 'or', 'were', 'housed', 'prior', 'to', '2000', 'when', 'the', 'housing', 'needs', 'assessment', 'categories', 'were', 'implemented', '.']
['In', 'reply', 'to', 'Ms', 'HILDYARD', '(', 'Reynell', ')', '(', '1', 'August', '2018', ')', '.']
['I', 'have', 'been', 'advised', ':', 'The', 'Hon', '.']
['That', 'Mr', 'Romaldi', 'has', 'not', 'been', 'remunerated', 'for', 'his', 'one', 'month', 'service', 'to', 'SAMEAC', '.']
['279', 'The', 'Hon', '.']
['1.', '&', '#', 'x9', ';', 'What', 'is', 'the', 'number', 'of', 'employed', 'FTE', 'positions', 'by', 'classification', 'level', '?']
['I', 'have', 'been', 'advised', ':', 'The', '

['(', 'v', ')', '&', '#', 'x9', ';', 'Neurosurgery', 'speciality', '?']
['(', 'w', ')', '&', '#', 'x9', ';', 'Ophthalmology', 'speciality', '?']
['(', 'x', ')', '&', '#', 'x9', ';', 'Orthopaedics', 'speciality', '?']
['(', 'y', ')', '&', '#', 'x9', ';', 'Paediatric', 'medicine', 'speciality', '?']
['(', 'z', ')', '&', '#', 'x9', ';', 'Paediatric', 'surgery', 'speciality', '?']
['(', 'aa', ')', '&', '#', 'x9', ';', 'Pain', 'management', 'speciality', '?']
['(', 'bb', ')', '&', '#', 'x9', ';', 'Palliative', 'care', 'speciality', '?']
['(', 'cc', ')', '&', '#', 'x9', ';', 'Plastic', 'and', 'reconstructive', 'surgery', 'speciality', '?']
['(', 'dd', ')', '&', '#', 'x9', ';', 'Psychiatry', 'speciality', '?']
['(', 'ee', ')', '&', '#', 'x9', ';', 'H', 'radiation', 'therapy', '(', 'consultation', ')', 'speciality', '?']
['(', 'ff', ')', '&', '#', 'x9', ';', 'Rehabilitation—medicine', 'speciality', '?']
['(', 'gg', ')', '&', '#', 'x9', ';', 'Respiratory', 'speciality', '?']
['(', 'hh', ')', '&

['5.', '&', '#', 'x9', ';', 'What', 'is', 'the', 'nature', 'of', 'the', 'work', 'they', 'undertake', '?']
['The', 'Attorney-General', 'has', 'provided', 'the', 'following', 'advice', ':', 'The', 'Hon', '.']
['1.', '&', '#', 'x9', ';', 'Efficiency', 'savings', 'may', 'come', 'from', 'staff', 'reductions', ',', 'and', 'other', 'areas', '.']
['2.', '&', '#', 'x9', ';', '2018-19', 'Budget', 'Paper', '5', 'sets', 'out', 'that', 'the', 'expected', 'full-time', 'equivalent', 'reduction', 'will', 'be', '15', '.']
['3.', '&', '#', 'x9', ';', 'Savings', 'relevant', 'to', 'staffing', 'will', 'likely', 'impact', 'on', 'all', 'areas', 'of', 'the', 'Crown', 'Solicitor', "'s", 'Office', '.']
['4.', '&', '#', 'x9', ';', 'This', 'detail', 'is', 'yet', 'to', 'be', 'finalised', '.']
['5.', '&', '#', 'x9', ';', 'This', 'detail', 'is', 'yet', 'to', 'be', 'finalised', '.']
['70', 'The', 'Hon', '.']
['1.', '&', '#', 'x9', ';', 'Is', 'the', 'projection', 'of', 'drug', 'treatment', 'assessments', 'increasing',

['2.', '&', '#', 'x9', ';', 'What', 'does', 'success', 'look', 'like', 'for', 'the', 'implementation', 'of', 'phase', '2', 'of', 'the', 'Gambling', 'Harm', 'Minimisation', 'Strategy', 'and', 'what', 'are', 'the', 'key', 'performance', 'indicators', 'of', 'the', 'strategy', '?']
['The', 'Minister', 'for', 'Human', 'Services', 'has', 'provided', 'the', 'following', 'advice', ':', 'The', 'Hon', '.']
['The', 'priority', 'for', 'the', 'government', 'in', 'implementing', 'phase', '2', 'of', 'the', 'Gambling', 'Harm', 'Minimisation', 'Strategy', 'is', 'to', 'ensure', 'that', 'patrons', 'from', 'diverse', 'backgrounds', 'are', 'aware', 'of', 'the', 'role', 'that', 'gaming', 'venue', 'staff', 'have', 'to', 'support', 'them', 'to', 'prevent', 'gambling', 'harm', '.']
['Engagement', 'between', 'venue', 'staff', 'and', 'patrons', 'can', 'encourage', 'individuals', 'to', 'seek', 'help', 'from', 'services', 'to', 'reduce', 'the', 'further', 'incidence', 'of', 'gambling', 'harm', 'occurring', '.']
['

['The', 'full', 'list', 'of', 'the', 'savings', 'strategies', 'of', 'the', 'government', ',', 'including', 'cuts', 'to', 'programs', 'and', 'initiatives', ',', 'are', 'set', 'out', 'in', 'detail', 'in', 'the', '2018-19', 'Budget', 'Measures', 'Statement', '.']
['The', 'only', 'adjustment', 'to', 'the', 'savings', 'strategies', 'set', 'out', 'in', 'the', 'budget', 'papers', 'has', 'been', 'a', 'reduction', 'of', '$', '23', 'million', 'in', 'the', '2018-19', 'savings', 'target', 'of', 'the', 'Health', 'portfolio', '.']
['624', 'Mr', 'ODENWALDER', '(', 'Elizabeth', ')', '(', '26', 'February', '2019', ')', '.']
['As', 'at', '31', 'December', '2018', 'the', 'total', 'number', '(', 'FTE', ')', 'of', 'police', 'cadets', 'in', 'South', 'Australia', 'was', '152.The', 'Hon', '.']
['682', 'Mr', 'ODENWALDER', '(', 'Elizabeth', ')', '(', '19', 'March', '2019', ')', '.']
['The', 'State', 'Procurement', 'Board', "'s", 'Procurement', 'Policy', 'Framework', 'sets', 'out', 'the', 'overarching', 'policy'

## Filter Stopwords in a tokenized sentence

In [32]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\xcelsior\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [33]:
english_stops = set(stopwords.words('english'))

In [39]:
listOfWordsWithoutStops = []
for listOfWords in wordTokens:
#     print(listOfWords)
        listOfWordsWithoutStopsInstance = [word for word in listOfWords if word not in english_stops]
        listOfWordsWithoutStops.append(listOfWordsWithoutStopsInstance)

In [40]:
listOfWordsWithoutStops

[['1.',
  '&',
  '#',
  'x9',
  ';',
  'I',
  'refer',
  'honourable',
  'member',
  'media',
  'release',
  'issued',
  '7',
  'March',
  '2019',
  'titled',
  "'",
  '$',
  '25',
  'Million',
  'fix',
  'required',
  'Ageing',
  'Dog',
  'Fence',
  '.',
  "'"],
 ['2.',
  '&',
  '#',
  'x9',
  ';',
  'In',
  'total',
  ',',
  '115',
  'kilometres',
  '2,150',
  'kilometre',
  'South',
  'Australian',
  'Dog',
  'Fence',
  'replaced',
  '16',
  'years',
  'former',
  'Labor',
  'government',
  '.'],
 ['3.',
  '&',
  '#',
  'x9',
  ';',
  'Based',
  'advice',
  'received',
  'failure',
  'electric',
  'fencing',
  'installed',
  'former',
  'Labor',
  'government',
  'I',
  'would',
  'envisage',
  'replacing',
  'sections',
  'Dog',
  'Fence',
  'electric',
  'fencing',
  '.'],
 ['In', 'reply', 'Hon', '.'],
 ['The',
  'Minister',
  'Transport',
  ',',
  'Infrastructure',
  'Local',
  'Government',
  'provided',
  'following',
  'advice',
  ':',
  'The',
  'Hon',
  '.'],
 ['Upkeep',
  '

## sysnets and hypernyms

In [41]:
from nltk.corpus import wordnet
syn = wordnet.synsets('police')[0]

In [42]:
syn.name()

'police.n.01'

In [43]:
syn.definition()

'the force of policemen and officers'

In [44]:
wordnet.synsets('police')[0].examples()

['the law came looking for him']

In [45]:
syn.hypernyms()

[Synset('force.n.04'), Synset('law_enforcement_agency.n.01')]

In [46]:
syn.hypernym_paths()

[[Synset('entity.n.01'),
  Synset('abstraction.n.06'),
  Synset('group.n.01'),
  Synset('social_group.n.01'),
  Synset('organization.n.01'),
  Synset('force.n.04'),
  Synset('police.n.01')],
 [Synset('entity.n.01'),
  Synset('abstraction.n.06'),
  Synset('group.n.01'),
  Synset('social_group.n.01'),
  Synset('organization.n.01'),
  Synset('unit.n.03'),
  Synset('administrative_unit.n.01'),
  Synset('agency.n.01'),
  Synset('law_enforcement_agency.n.01'),
  Synset('police.n.01')]]

### Part of speech Tag
Noun = n

Adjective = a

Adverb = r

Verb = v

In [48]:
syn.pos()

'n'

## Lemmas and synonyms
#### A lemma (in linguistics), is the canonical form or morphological form of a word.

In [51]:
lemmas = syn.lemmas()
loops = len(lemmas)

In [52]:
for i in range(loops):
    print(lemmas[i].name())

police
police_force
constabulary
law


In [53]:
lemmas[0].synset() == lemmas[1].synset()

True

In [59]:
synonyms = []
for syn in wordnet.synsets('politician'):
    for lemma in syn.lemmas():
        synonyms.append(lemma.name())
len(synonyms)

6

In [60]:
synonyms

['politician',
 'politician',
 'politico',
 'pol',
 'political_leader',
 'politician']

## Antonyms

In [65]:
gn2 = wordnet.synset('good.n.02')
gn2.definition()

'moral excellence or admirableness'

In [66]:
evil = gn2.lemmas()[0].antonyms()[0]

In [68]:
evil.name
evil.synset().definition()

'the quality of being morally wrong in principle or practice'

In [69]:
ga1 = wordnet.synset('good.a.01')
ga1.definition()

'having desirable or positive qualities especially those suitable for a thing specified'

In [70]:
bad = ga1.lemmas()[0].antonyms()[0]
bad.name()

'bad'

In [71]:
bad.synset().definition()

'having undesirable or negative qualities'

## Calculating Similarity - Wu-Palmer Similarity

In [72]:
from nltk.corpus import wordnet


In [74]:
cb = wordnet.synset('police.n.01')

In [75]:
ib = wordnet.synset('constabulary.n.01')

In [76]:
cb.wup_similarity(ib)

0.9

In [77]:
ref = cb.hypernyms()[0]

In [78]:
cb.shortest_path_distance(ref)

1

In [79]:
ib.shortest_path_distance(ref)

1

In [80]:
cb.shortest_path_distance(ib)

0

## Discover word collocations - (need to run pre stopwords) - also need to pick a larger text blob to get themes for the day

In [82]:
from nltk.corpus import webtext
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures

In [ ]:
words = [w.lower() for w in webtext.words('grail.txt')]

In [91]:
# pre stop words
listOfWordsWithoutStopsBigramCollocationFinder = []
for listOfWords in sentTokens:
#     print(listOfWords)
        listOfWordsWithoutStopsInstance = [word for word in listOfWords if word not in english_stops]
        bcf = BigramCollocationFinder.from_words(listOfWordsWithoutStopsInstance)
        listOfWordsWithoutStopsInstance =  bcf.nbest(BigramAssocMeasures.likelihood_ratio, 4)
        
        
        listOfWordsWithoutStopsBigramCollocationFinder.append(listOfWordsWithoutStopsInstance)
listOfWordsWithoutStopsBigramCollocationFinder

[[],
 [('2.&#x9;In total, 115 kilometres of the 2,150 kilometre South Australian Dog Fence was replaced over 16\xa0years under the former Labor government.',
   "Of this, 68.5 kilometres is electric fence, which now needs to be replaced because it is too low and because pastoralists don't believe it is effective.")],
 [],
 [('In reply to the Hon.', 'J.A.'), ('J.A.', 'DARLEY (5 December 2018).')],
 [('  The Minister for Transport, Infrastructure and Local Government has provided the following advice:The Hon.',
   'D.W. RIDGWAY (Minister for Trade, Tourism and Investment):  The Minister for Transport, Infrastructure and Local Government has provided the following advice:')],
 [('Upkeep and maintenance of rail infrastructure remains a key safety requirement and where required, substitute bus services are offered as replacements for train services during closures.',
   'Advance notice regarding these works and the deployment of substitute buses is advertised via the Adelaide Metro website,

In [92]:
# post stop words
listOfWordsWithoutStopsBigramCollocationFinder = []
for listOfWords in listOfWordsWithoutStops:
#     print(listOfWords)
        listOfWordsWithoutStopsInstance = [word for word in listOfWords if word not in english_stops]
        bcf = BigramCollocationFinder.from_words(listOfWordsWithoutStopsInstance)
        listOfWordsWithoutStopsInstance =  bcf.nbest(BigramAssocMeasures.likelihood_ratio, 4)
        listOfWordsWithoutStopsBigramCollocationFinder.append(listOfWordsWithoutStopsInstance)
listOfWordsWithoutStopsBigramCollocationFinder

[[('#', 'x9'), ('$', '25'), ('&', '#'), ('1.', '&')],
 [('#', 'x9'), ('&', '#'), (',', '115'), ('115', 'kilometres')],
 [('electric', 'fencing'), ('#', 'x9'), ('&', '#'), ('3.', '&')],
 [('Hon', '.'), ('In', 'reply'), ('reply', 'Hon')],
 [(',', 'Infrastructure'),
  ('Government', 'provided'),
  ('Hon', '.'),
  ('Infrastructure', 'Local')],
 [(',', 'substitute'),
  ('Upkeep', 'maintenance'),
  ('closures', '.'),
  ('infrastructure', 'remains')],
 [('These', 'communication'),
  ('advanced', 'choice'),
  ('bus', 'service'),
  ('choice', 'whether')],
 [("'", 'patience'),
  (',', 'Transport'),
  ('Australian', 'government'),
  ('Department', 'Planning')],
 [("'s", 'whereby'),
  ('Ageing', 'Well'),
  ('Card', 'Unit'),
  ('Office', 'Ageing')],
 [('Adelaide', 'Metro'),
  ('Metro', 'website'),
  ('Metrocard', 'policies'),
  ('also', 'available')],
 [('3', 'April'), ('701', 'Mr'), ('April', '2019'), ('Mr', 'PICTON')],
 [('Health', 'Wellbeing'),
  ('Hon', '.'),
  ('Minister', 'Health'),
  ('Wellb

In [112]:
from nltk.collocations import TrigramCollocationFinder
from nltk.metrics import TrigramAssocMeasures

from nltk.corpus import stopwords
stopset = set(stopwords.words('english'))
filter_stops = lambda w: len(w) < 3 or w in stopset
bcf.apply_word_filter(filter_stops)
bcf.nbest(BigramAssocMeasures.likelihood_ratio, 4)

[('Development', 'advised'),
 ('Industries', 'Regional'),
 ('Minister', 'Primary'),
 ('Primary', 'Industries')]

## TrigramCollocationFinder

In [122]:
from nltk.collocations import TrigramCollocationFinder
from nltk.metrics import TrigramAssocMeasures

allWords = []
loops = len(listOfWordsWithoutStops)

for i in range(loops):
    words = [w.lower() for w in listOfWordsWithoutStops[i]]
    allWords.append(words[0])




In [240]:
allWords

['1.',
 '2.',
 '3.',
 'in',
 'the',
 'upkeep',
 'these',
 'our',
 'metrocard',
 'information',
 '701',
 'the',
 'planning',
 'in',
 'the',
 '1.',
 'b',
 'b',
 '2.',
 'one',
 'six',
 'ten',
 '19',
 'b',
 '658',
 '1.',
 '2.',
 '3.',
 'i',
 'table',
 'table',
 'b',
 'b',
 'note',
 'shared',
 '707',
 'the',
 'the',
 'in',
 'the',
 '1.',
 '2.',
 'any',
 'any',
 'development',
 'development',
 'development',
 'development',
 '3.',
 '4.',
 'in',
 '8.',
 '705',
 'the',
 'the',
 'b',
 'b',
 '702',
 'the',
 'it',
 '693',
 'the',
 'ratios',
 '699',
 'the',
 'negotiations',
 'in',
 'i',
 'i',
 'during',
 'in',
 'i',
 '700',
 '(',
 '(',
 '(',
 '(',
 'the',
 '(',
 '(',
 '(',
 '(',
 '697',
 'the',
 'the',
 '711',
 'the',
 'ward',
 'in',
 'the',
 'the',
 '659',
 '1.',
 '2.',
 'i',
 'information',
 'table',
 'b',
 'b',
 'shared',
 'in',
 'the',
 'the',
 'dr',
 '703',
 'the',
 'i',
 'in',
 'i',
 '1.',
 'displayed',
 'the',
 'flyers',
 'customer',
 'information',
 'my',
 'regular',
 'timetable',
 'voice'

In [125]:
tcf = TrigramCollocationFinder.from_words(allWords)
tcf.apply_word_filter(filter_stops)
tcf.apply_freq_filter(3)
tcf.nbest(TrigramAssocMeasures.likelihood_ratio, 4)
    

[('creation', 'refurbishment', 'landscaping'),
 ('removal', 'creation', 'refurbishment'),
 ('10.', '11.', '12.'),
 ('13.', '14.', '15.')]

### Stemming words

### Lemmatizing words with WordNet

### Replacing words matching regular expressions

### Removing repeating characters

### Spelling correction with Enchant

### Replacing synonyms

### Replacing negations with antonyms

In [126]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
stemmer.stem('police')

'polic'

In [128]:
stemmer.stem('politician')

'politician'

## Lancaster stemming algorithm

In [130]:
from nltk.stem import LancasterStemmer
stemmer = LancasterStemmer()
stemmer.stem('running')

'run'

In [131]:
stemmer.stem('runner')

'run'

## RegexpStemmer

In [134]:
from nltk.stem import RegexpStemmer
stemmer = RegexpStemmer('ing')
stemmer.stem('running')

'runn'

In [135]:
stemmer.stem('runner')

'runner'

In [136]:
stemmer.stem('Adelaide')

'Adelaide'

## SnowballStemmer

In [140]:
from nltk.stem import SnowballStemmer

spanish_stemmer = SnowballStemmer('english')
spanish_stemmer.stem('howdy')

'howdi'

## Lemmatizing words with WordNet

In [141]:
from nltk.stem import WordNetLemmatizer


In [144]:
lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize('running')

'running'

In [145]:
lemmatizer.lemmatize('running', pos='v')

'run'

In [147]:
lemmatizer.lemmatize('runners')

'runner'

## important example for difference between stemming and lemmatization 

In [151]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
stemmer.stem('believes')

'believ'

In [150]:
lemmatizer.lemmatize('believes')


'belief'

## Combining stemming with lemmatization

In [164]:
stemmer.stem('buses')

'buse'

In [165]:
lemmatizer.lemmatize('buses')


'bus'

In [166]:
stemmer.stem('bus')

'bu'

stemming saves one character, lemmatization saves two characters, and
stemming the lemma saves a total of three characters out of five characters. That is nearly
a 60% compression rate! This level of word compression over many thousands of words,
while unlikely to always produce such high gains, can still make a huge difference.

## spell correct 

In [178]:
# import enchant
from nltk.metrics import edit_distance
# class SpellingReplacer(object):
#     def __init__(self, dict_name='en', max_dist=2):
#         self.spell_dict = enchant.Dict(dict_name)
#         self.max_dist = max_dist
    
#     def replace(self, word):
#         if self.spell_dict.check(word):
#             return word
#         suggestions = self.spell_dict.suggest(word)
    
#         if suggestions and edit_distance(word, suggestions[0]) <= self.max_dist:
#             return suggestions[0]
#         else:
#             return word

In [179]:
from replacers import SpellingReplacer
# replacer = SpellingReplacer()
# replacer.replace('word')

ModuleNotFoundError: No module named 'replacers'

## Replacing synonyms

In [180]:
class WordReplacer(object):
    def __init__(self, word_map):
        self.word_map = word_map
    def replace(self, word):
        return self.word_map.get(word, word)

In [182]:
replacer = WordReplacer({'sapol': 'police'})
replacer.replace('sapol')


'police'

# csv replacement


In [187]:
# requires a csv with 2 columns 
# word     |    synonym 
# police   |     sapol

In [185]:
import csv
class CsvWordReplacer(WordReplacer):
    def __init__(self, fname):
        word_map = {}
        for line in csv.reader(open(fname)):
            word, syn = line
            word_map[word] = syn
        super(CsvWordReplacer, self).__init__(word_map)

# replace words with  Antonym 

In [190]:
from nltk.corpus import wordnet
class AntonymReplacer(object):
    def replace(self, word, pos=None):
        antonyms = set()
        for syn in wordnet.synsets(word, pos=pos):
            for lemma in syn.lemmas():
                for antonym in lemma.antonyms():
                    antonyms.add(antonym.name())
        if len(antonyms) == 1:
            return antonyms.pop()
        else:
            return None
        
    def replace_negations(self, sent):
        i, l = 0, len(sent)
        words = []
        while i < l:
            word = sent[i]
            if word == 'not' and i+1 < l:
                ant = self.replace(sent[i+1])
                if ant:
                    words.append(ant)
                    i += 2
                    continue
            words.append(word)
            i += 1
        return words

In [198]:
# from replacers import AntonymReplacer
replacer = AntonymReplacer()
replacer.replace('nice')

'nasty'

In [206]:
replacer.replace('uglify')

'beautify'

In [215]:
sent = ["lets", 'not', 'uglify', 'america', 'great', 'again']

In [216]:
replacer.replace_negations(sent)

['lets', 'beautify', 'america', 'great', 'again']

In [219]:
class AntonymWordReplacer(WordReplacer, AntonymReplacer):
    pass

In [220]:
replacer = AntonymWordReplacer({'evil': 'good'})
replacer.replace_negations(['good', 'is', 'not', 'evil'])

['good', 'is', 'good']

# Creating the Corpora

In [233]:
import os, os.path
path = os.path.expanduser('~/nltk_data')

if not os.path.exists(path):
    os.mkdir(path)

os.path.exists(path)

import nltk.data
path in nltk.data.path

True

In [237]:
from nltk.corpus.reader import WordListCorpusReader

In [329]:
reader = WordListCorpusReader(".",wordTokens)

In [330]:
a = list(reader.fileids())

In [342]:
with open('words.txt', 'w') as f:
    for item in a:
        print(item)
        f.write("%s\n" % item)
    f.close

['1.', '&', '#', 'x9', ';', 'I', 'refer', 'the', 'honourable', 'member', 'to', 'the', 'media', 'release', 'issued', 'on', '7', 'March', '2019', 'titled', "'", '$', '25', 'Million', 'fix', 'required', 'for', 'Ageing', 'Dog', 'Fence', '.', "'"]
['2.', '&', '#', 'x9', ';', 'In', 'total', ',', '115', 'kilometres', 'of', 'the', '2,150', 'kilometre', 'South', 'Australian', 'Dog', 'Fence', 'was', 'replaced', 'over', '16', 'years', 'under', 'the', 'former', 'Labor', 'government', '.']
['3.', '&', '#', 'x9', ';', 'Based', 'on', 'the', 'advice', 'received', 'about', 'the', 'failure', 'of', 'the', 'electric', 'fencing', 'installed', 'under', 'the', 'former', 'Labor', 'government', 'I', 'would', 'not', 'envisage', 'replacing', 'sections', 'of', 'the', 'Dog', 'Fence', 'with', 'electric', 'fencing', '.']
['In', 'reply', 'to', 'the', 'Hon', '.']
['The', 'Minister', 'for', 'Transport', ',', 'Infrastructure', 'and', 'Local', 'Government', 'has', 'provided', 'the', 'following', 'advice', ':', 'The', 'Ho

['The', 'Minister', 'for', 'Transport', ',', 'Infrastructure', 'and', 'Local', 'Government', 'has', 'provided', 'the', 'following', 'advice', ':', 'The', 'Hon', '.']
['SeaLink', "'s", 'current', 'licence', 'to', 'operate', 'a', 'ferry', 'service', 'expires', 'in', 'July', '2024', '.']
['The', 'tender', 'assessment', 'criteria', 'will', 'be', 'outlined', 'in', 'future', 'tender', 'documents', '.']
['The', 'department', 'will', 'fulfil', 'its', 'obligation', 'under', 'the', 'Department', 'of', 'the', 'Premier', 'and', 'Cabinet', 'Circular', 'PC27—Disclosure', 'of', 'Government', 'Contracts', '.']
['In', 'reply', 'to', 'the', 'Hon', '.']
['The', 'Minister', 'for', 'Police', ',', 'Emergency', 'Services', 'and', 'Correctional', 'Services', 'has', 'advised', ':', 'The', 'Hon', '.']
['The', 'meeting', 'brought', 'together', 'key', 'industry', 'stakeholders', ',', 'including', 'industry', 'experts', ',', 'to', 'address', 'road', 'safety', 'in', 'South', 'Australia', '.']
['Attendees', 'of', 't

UnicodeEncodeError: 'charmap' codec can't encode character '\u2265' in position 273: character maps to <undefined>

In [346]:
from nltk.tokenize import line_tokenize
# line_tokenize(reader)

In [347]:
import nltk
nltk.download('names')
nltk.download('words')

[nltk_data] Downloading package names to
[nltk_data]     C:\Users\xcelsior\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\xcelsior\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [348]:
from nltk.corpus import names
names.fileids()



['female.txt', 'male.txt']

In [349]:
from nltk.corpus import words
words.fileids()

['en', 'en-basic']

In [350]:
from nltk.corpus.reader import TaggedCorpusReader

In [351]:
reader = TaggedCorpusReader('.', r'.*\.txt')

In [352]:
reader

<TaggedCorpusReader in 'C:\\Users\\xcelsior\\Desktop\\personal\\uni\\2019 uni\\Sem2 Data Science Professional Development'>

In [353]:
reader.tagged_words()

[('root', None), ('|--', None), ('hansard:', None), ...]

In [354]:
reader.sents()

[['root'], ['|--', 'hansard:', 'struct', '(nullable', '=', 'true)'], ...]

In [341]:
reader.tagged_sents()

[[('root', None)], [('|--', None), ('hansard:', None), ('struct', None), ('(nullable', None), ('=', None), ('true)', None)], ...]

In [322]:
# reader.paras()

In [325]:
# reader.tagged_paras()

In [355]:
from nltk.tokenize import SpaceTokenizer
reader = TaggedCorpusReader('.', r'.*\.txt', word_tokenizer=SpaceTokenizer())
reader.words()

['root\r', '', '|--', 'hansard:', 'struct', ...]

In [357]:
from nltk.tokenize import LineTokenizer
reader = TaggedCorpusReader('.', r'.*\.txt', sent_tokenizer=LineTokenizer())
reader.sents()

[['root'], ['|--', 'hansard:', 'struct', '(nullable', '=', 'true)'], ...]

# universal tagset

In [360]:
import nltk
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\xcelsior\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\universal_tagset.zip.


True

In [363]:
reader = TaggedCorpusReader('.', r'.*\.txt', tagset='en-brown')
kk = reader.tagged_words(tagset='universal')